In [1]:

from dotenv import load_dotenv
import duckdb
from pathlib import Path

load_dotenv()

True

In [74]:
base_path = Path().cwd().parent
source_1_path = base_path / Path('silver/anilist/voice-actor/voice-actor-2024-1-15.parquet')
source_2_path = base_path / Path('silver/anilist/staff/staff-2024-1-16.parquet')
target_path = base_path / Path('gold/anilist/dim_people.parquet')

In [34]:
tb_va = duckdb.read_parquet(str(source_1_path))
tb_staff = duckdb.read_parquet(str(source_2_path))

In [35]:
tb_va.shape

(62, 5)

In [79]:
tb_staff.shape

(51710, 26)

In [71]:
tb_dim = duckdb.sql("""
WITH source_va AS (
    SELECT 
        id AS people_id
        , name_first
        , name_last
    FROM tb_va  
), source_staff AS (
    SELECT
        staff_id AS people_id
        , name_first
        , name_middle
        , name_last
        , name_full
        , name_native
        , name_alternative
        , language
        , image_medium
        , description
        , primaryOccupations
        , gender
        , make_date(dateOfBirth_year, dateOfBirth_month, dateOfBirth_day) AS dateOfBirth
        --, make_date(dateOfDeath_year, dateOfDeath_month, dateOfDeath_day) AS dateOfDeath  -- Discard due to data quality
        , age
        , yearsActive
        , homeTown
        , bloodType
        , siteUrl
        FROM tb_staff
), source_union AS (
    SELECT * FROM source_va UNION ALL BY NAME SELECT * FROM source_staff
), dedupe AS (
    SELECT
        people_id
        , ANY_VALUE(name_first) AS name_first
        , ANY_VALUE(name_middle) AS name_middle
        , ANY_VALUE(name_last) AS name_last
        , ANY_VALUE(name_full) AS name_full
        , ANY_VALUE(name_native) AS name_native
        , ANY_VALUE(name_alternative) AS name_alternative
        , ANY_VALUE(language) AS language
        , ANY_VALUE(image_medium) AS image_medium
        , ANY_VALUE(description) AS description
        , ANY_VALUE(primaryOccupations) AS primaryOccupations
        , ANY_VALUE(gender) AS gender
        , ANY_VALUE(dateOfBirth) AS dateOfBirth
        , ANY_VALUE(age) AS age
        , ANY_VALUE(yearsActive) AS yearsActive
        , ANY_VALUE(homeTown) AS homeTown
        , ANY_VALUE(bloodType) AS bloodType
        , ANY_VALUE(siteUrl) AS siteUrl
    FROM source_union
    GROUP BY (people_id)
)
    
    SELECT * FROM dedupe

""")
# SELECT DISTINCT v.id, s.staff_id, v.name_first, s.name_first
# FROM tb_va v FULL JOIN tb_staff s ON v.id=s.staff_id

In [76]:
tb_dim.shape

(14345, 18)

In [75]:
tb_dim.to_parquet(str(target_path))